In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import layers, models
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('df_anomaly.csv')


df = df[['Месяц', 'Адрес объекта', 'dt', '№ ОДПУ', 'Тип объекта',
       'Дата текущего показания', 'Текущее потребление, Гкал', 'avg_temp',
       'days', 'ГВС-ИТП', 'Общая площадь объекта',
       'year_range_1959-1989', 'year_range_1990-2000', 'year_range_2001-2010',
       'year_range_2011-2024', 'Number_of_storeys_range_3-4',
       'Number_of_storeys_range_5-9', 'Number_of_storeys_range_10-12',
       'Number_of_storeys_range_> 13', 'anomaly']]

In [21]:
#df = df.set_index(['dt','№ ОДПУ','Адрес объекта'])
df.head()

Месяц  Тип объекта  \
dt         № ОДПУ  Адрес объекта                                     
2022-01-01 147256  ул. Петергофская, д.2            1    -1.613207   
           1740725 ул. Кирова, д.103                1     0.557138   
           247368  ул. Трамвайная, д.13 корп.1      1    -1.613207   
           98735   ул. Достоевского, д.112          1     0.557138   
           28884   ул. Рудольфа Нуреева, д.21       1     0.557138   

                                                Дата текущего показания  \
dt         № ОДПУ  Адрес объекта                                          
2022-01-01 147256  ул. Петергофская, д.2                      -0.990352   
           1740725 ул. Кирова, д.103                          -0.990352   
           247368  ул. Трамвайная, д.13 корп.1                -0.990352   
           98735   ул. Достоевского, д.112                    -0.990352   
           28884   ул. Рудольфа Нуреева, д.21                 -0.990352   

                                                Текущее потребление, Гкал  \
dt         № ОДПУ  Адрес объекта                                            
2022-01-01 147256  ул. Петергофская, д.2                           24.472   
           1740725 ул. Кирова, д.103                               74.652   
           247368  ул. Трамвайная, д.13 корп.1                     20.781   
           98735   ул. Достоевского, д.112                        167.411   
           28884   ул. Рудольфа Нуреева, д.21                     282.680   

                                                avg_temp  days  ГВС-ИТП  \
dt         № ОДПУ  Адрес объекта                                          
2022-01-01 147256  ул. Петергофская, д.2           -11.9    31    False   
           1740725 ул. Кирова, д.103               -11.9    31    False   
           247368  ул. Трамвайная, д.13 корп.1     -11.9    31    False   
           98735   ул. Достоевского, д.112         -11.9    31    False   
           28884   ул. Рудольфа Нуреева, д.21      -11.9    31     True   

                                                Общая площадь объекта  \
dt         № ОДПУ  Адрес объекта                                        
2022-01-01 147256  ул. Петергофская, д.2                          NaN   
           1740725 ул. Кирова, д.103                         0.079531   
           247368  ул. Трамвайная, д.13 корп.1                    NaN   
           98735   ул. Достоевского, д.112                  -0.139965   
           28884   ул. Рудольфа Нуреева, д.21                1.462582   

                                               year_range_1959-1989  \
dt         № ОДПУ  Адрес объекта                                      
2022-01-01 147256  ул. Петергофская, д.2                        NaN   
           1740725 ул. Кирова, д.103                           True   
           247368  ул. Трамвайная, д.13 корп.1                  NaN   
           98735   ул. Достоевского, д.112                     True   
           28884   ул. Рудольфа Нуреева, д.21                 False   

                                               year_range_1990-2000  \
dt         № ОДПУ  Адрес объекта                                      
2022-01-01 147256  ул. Петергофская, д.2                        NaN   
           1740725 ул. Кирова, д.103                          False   
           247368  ул. Трамвайная, д.13 корп.1                  NaN   
           98735   ул. Достоевского, д.112                    False   
           28884   ул. Рудольфа Нуреева, д.21                 False   

                                               year_range_2001-2010  \
dt         № ОДПУ  Адрес объекта                                      
2022-01-01 147256  ул. Петергофская, д.2                        NaN   
           1740725 ул. Кирова, д.103                          False   
           247368  ул. Трамвайная, д.13 корп.1                  NaN   
           98735   ул. Достоевского, д.112                    False   
           28884   ул. Рудольфа

In [22]:
df = df.dropna()

In [49]:
#df['Дата текущего показания'] = pd.to_datetime(df['Дата текущего показания'])
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 68502 entries, ('2022-01-01', '1740725', 'ул. Кирова, д.103') to ('2022-12-01', '1981', 'р-н Уфимский, д Лекаревка')
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Месяц                          68502 non-null  int64  
 1   Тип объекта                    68502 non-null  float64
 2   Дата текущего показания        68502 non-null  float64
 3   Текущее потребление, Гкал      68502 non-null  float64
 4   avg_temp                       68502 non-null  float64
 5   days                           68502 non-null  int64  
 6   ГВС-ИТП                        68502 non-null  int64  
 7   Общая площадь объекта          68502 non-null  float64
 8   year_range_1959-1989           68502 non-null  int64  
 9   year_range_1990-2000           68502 non-null  int64  
 10  year_range_2001-2010           68502 non-null  int64  
 11  year_range_2011

In [37]:
le = LabelEncoder()

list_str = ['Тип объекта','Дата текущего показания','year_range_1959-1989','year_range_1990-2000','year_range_2001-2010',
           'year_range_2011-2024','Number_of_storeys_range_3-4','Number_of_storeys_range_5-9','Number_of_storeys_range_10-12',
           'Number_of_storeys_range_> 13','ГВС-ИТП']
for c in list_str:
    df[c] = le.fit_transform(df[c])

In [39]:

std_scaler = StandardScaler()

cols = list(df[['Тип объекта','Общая площадь объекта','Дата текущего показания']])
scaler = StandardScaler()
df[cols] = scaler.fit_transform(df[cols])

In [50]:
X = df.drop('anomaly', axis = 1)
y = df['anomaly']
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, random_state = 0)

X.head()

Месяц  Тип объекта  \
dt         № ОДПУ   Адрес объекта                                    
2022-01-01 1740725  ул. Кирова, д.103               1     0.551505   
           98735    ул. Достоевского, д.112         1     0.551505   
           28884    ул. Рудольфа Нуреева, д.21      1     0.551505   
           00277185 ул. Калинина, д.43              1     0.551505   
           69203    б-р. Тухвата Янаби, д.73        1     0.551505   

                                                Дата текущего показания  \
dt         № ОДПУ   Адрес объекта                                         
2022-01-01 1740725  ул. Кирова, д.103                          0.016683   
           98735    ул. Достоевского, д.112                    0.016683   
           28884    ул. Рудольфа Нуреева, д.21                 0.016683   
           00277185 ул. Калинина, д.43                         0.016683   
           69203    б-р. Тухвата Янаби, д.73                   0.016683   

                                                Текущее потребление, Гкал  \
dt         № ОДПУ   Адрес объекта                                           
2022-01-01 1740725  ул. Кирова, д.103                              74.652   
           98735    ул. Достоевского, д.112                       167.411   
           28884    ул. Рудольфа Нуреева, д.21                    282.680   
           00277185 ул. Калинина, д.43                             31.417   
           69203    б-р. Тухвата Янаби, д.73                       13.410   

                                                avg_temp  days  ГВС-ИТП  \
dt         № ОДПУ   Адрес объекта                                         
2022-01-01 1740725  ул. Кирова, д.103              -11.9    31        0   
           98735    ул. Достоевского, д.112        -11.9    31        0   
           28884    ул. Рудольфа Нуреева, д.21     -11.9    31        1   
           00277185 ул. Калинина, д.43             -11.9    31        0   
           69203    б-р. Тухвата Янаби, д.73       -11.9    31        0   

                                                Общая площадь объекта  \
dt         № ОДПУ   Адрес объекта                                       
2022-01-01 1740725  ул. Кирова, д.103                        0.079531   
           98735    ул. Достоевского, д.112                 -0.139965   
           28884    ул. Рудольфа Нуреева, д.21               1.462582   
           00277185 ул. Калинина, д.43                      -0.726110   
           69203    б-р. Тухвата Янаби, д.73                -0.764124   

                                                year_range_1959-1989  \
dt         № ОДПУ   Адрес объекта                                      
2022-01-01 1740725  ул. Кирова, д.103                              1   
           98735    ул. Достоевского, д.112                        1   
           28884    ул. Рудольфа Нуреева, д.21                     0   
           00277185 ул. Калинина, д.43                             0   
           69203    б-р. Тухвата Янаби, д.73                       1   

                                                year_range_1990-2000  \
dt         № ОДПУ   Адрес объекта                                      
2022-01-01 1740725  ул. Кирова, д.103                              0   
           98735    ул. Достоевского, д.112                        0   
           28884    ул. Рудольфа Нуреева, д.21                     0   
           00277185 ул. Калинина, д.43                             0   
           69203    б-р. Тухвата Янаби, д.73                       0   

                                                year_range_2001-2010  \
dt         № ОДПУ   Адрес объекта                                      
2022-01-01 1740725  ул. Кирова, д.103                              0   
           98735    ул. Достоевского, д.112                        0   
           28884    ул. Рудольфа Нуреева, д.21                     1   
           00277185 ул. Калинина, д.43                             0   
           

In [57]:
model = Sequential([
    Input(shape=(16,)),
    Dense(32, activation = 'relu', input_shape = (xtrain.shape[1],)),
    Dropout(0.1),
    Dense(32, activation = 'relu'),
    Dropout(0.5),
    Dense(1, activation = 'sigmoid')
])

In [58]:

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 32)                544       
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_34 (Dense)            (None, 32)                1056      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_35 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,633
Trainable params: 1,633
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.fit(xtrain, ytrain, epochs = 5, batch_size = 8, validation_data = (xtest, ytest))


Epoch 1/5
6851/6851 [==============================] - 20s 3ms/step - loss: 0.1695 - accuracy: 0.9516 - val_loss: 0.1572 - val_accuracy: 0.9532
Epoch 2/5
6851/6851 [==============================] - 19s 3ms/step - loss: 0.1649 - accuracy: 0.9515 - val_loss: 0.1570 - val_accuracy: 0.9532
Epoch 3/5
6851/6851 [==============================] - 19s 3ms/step - loss: 0.1629 - accuracy: 0.9512 - val_loss: 0.1540 - val_accuracy: 0.9533
Epoch 4/5
6851/6851 [==============================] - 18s 3ms/step - loss: 0.1608 - accuracy: 0.9518 - val_loss: 0.1552 - val_accuracy: 0.9533
Epoch 5/5
6851/6851 [==============================] - 18s 3ms/step - loss: 0.1591 - accuracy: 0.9518 - val_loss: 0.1520 - val_accuracy: 0.9533


In [61]:
loss, accuracy = model.evaluate(xtest, ytest)

print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

429/429 [==============================] - 1s 2ms/step - loss: 0.1520 - accuracy: 0.9533
Test loss: 0.1520
Test accuracy: 0.9533
